In [10]:
import sys
print(sys.executable)


C:\ProgramData\Anaconda3\python.exe


In [4]:
!pip install --user textblob

  Using cached textblob-0.19.0-py3-none-any.whl (624 kB)
  Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rvr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rvr\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
!pip install textblob

  Using cached textblob-0.19.0-py3-none-any.whl (624 kB)
  Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
  Attempting uninstall: nltk
    Found existing installation: nltk 3.6.5
    Uninstalling nltk-3.6.5:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\nltk-3.6.5-py3.10.egg-info\\dependency_links.txt'
Consider using the `--user` option or check the permissions.



In [7]:
import textblob.download_corpora
textblob.download_corpora.download_all()

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\rvr\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\rvr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rvr\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\rvr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\rvr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\rvr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


In [1]:
import pandas as pd
from textblob import TextBlob
import nltk

# Download necessary NLTK data (run this once)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/brown')
except LookupError:
    nltk.download('brown')

class PRUSRecommender:
    def __init__(self, data):
        """
        Initialize with a DataFrame containing 'Product' and 'Review' columns.
        """
        self.data = data
        self.product_features = {} # Stores counts: {product: {feature: {'pos': 0, 'neg': 0}}}
        
    def process_reviews(self):
        """
        Phase 1 & 2: Split reviews into sentences, extract features and sentiments.
        Ref: Section III-B-2 of the paper.
        """
        print("Processing reviews... (This may take a moment for large datasets)")
        
        for index, row in self.data.iterrows():
            product = row['Product']
            review_text = str(row['Review'])
            
            if product not in self.product_features:
                self.product_features[product] = {}
            
            # Step 1: Break down into sentences 
            blob = TextBlob(review_text)
            for sentence in blob.sentences:
                # Step 2: Sentiment Analysis & Feature Extraction 
                sentiment = sentence.sentiment.polarity
                features = sentence.noun_phrases
                
                # Determine polarity category
                # Using a small threshold to decide positive/negative
                if sentiment > 0.05:
                    polarity = 'pos'
                elif sentiment < -0.05:
                    polarity = 'neg'
                else:
                    continue # Skip neutral for ranking dominance as per some implementations, or treat as neutral
                
                # Append extracted features with sentiments [cite: 21]
                for feature in features:
                    feature = feature.lower()
                    if feature not in self.product_features[product]:
                        self.product_features[product][feature] = {'pos': 0, 'neg': 0}
                    
                    self.product_features[product][feature][polarity] += 1

    def rank_ify(self, query_features, w1=0.5, w2=0.5):
        """
        Phase 3: RANK-ify Algorithm.
        Ranks products based on user specifications and sentiment weights.
        w1: Weight for positive sentiment
        w2: Weight for negative sentiment (w1 + w2 = 1)
        Ref: Algorithm 1 & Equation 1 in the paper.
        """
        ranked_products = []
        
        # Normalize weights if they don't sum to 1 [cite: 228]
        if w1 + w2 != 1:
            total_w = w1 + w2
            w1 = w1 / total_w
            w2 = w2 / total_w
            
        print(f"\nRanking for features: {query_features} (w1={w1}, w2={w2})")
        
        for product, features_data in self.product_features.items():
            rank_score = 0
            found_any = False
            
            for user_feature in query_features:
                user_feature = user_feature.lower()
                
                # Check if product has this feature (exact match or substring match)
                # In practice, fuzzy matching might be better, but strict match is faster
                matched_db_features = [f for f in features_data.keys() if user_feature in f]
                
                for db_f in matched_db_features:
                    stats = features_data[db_f]
                    c_pos = stats['pos']
                    c_neg = stats['neg']
                    total_count = c_pos + c_neg
                    
                    if total_count > 0:
                        # Equation 1: FeaSco(f) 
                        # FeaSco = (w1 * c(f+) - w2 * c(f-)) / (c(f+) + c(f-))
                        fea_sco = (w1 * c_pos - w2 * c_neg) / total_count
                        rank_score += fea_sco
                        found_any = True
            
            if found_any:
                ranked_products.append((product, rank_score))
        
        # Sort by Rank Score descending [cite: 231]
        ranked_products.sort(key=lambda x: x[1], reverse=True)
        return ranked_products

# --- Usage Example ---

# 1. Load Data (Replace this with pd.read_excel('your_file.xlsx'))
data = {
    'Product': ['Phone A', 'Phone A', 'Phone B', 'Phone B', 'Phone C'],
    'Review': [
        "The battery life is amazing and the screen is bright.",
        "Bad camera quality but good screen.",
        "The battery drains fast. Terrible experience.",
        "Camera is decent, but battery is bad.",
        "Excellent camera and battery life is good too."
    ]
}
df = pd.DataFrame(data)

# 2. Initialize System
prus = PRUSRecommender(df)

# 3. Process Reviews (Extract features/sentiments)
prus.process_reviews()

# 4. User Query & Ranking
# User wants 'battery' and 'screen'. 
# Let's say user cares more about avoiding negatives (higher w2) or balanced (0.5/0.5).
user_query = ['battery', 'screen']
results = prus.rank_ify(user_query, w1=0.5, w2=0.5)

# 5. Output Results
print("\nRecommended Product List:")
print(f"{'Product':<15} | {'Rank Score':<10}")
print("-" * 30)
for p, score in results:
    print(f"{p:<15} | {score:.4f}")

Processing reviews... (This may take a moment for large datasets)

Ranking for features: ['battery', 'screen'] (w1=0.5, w2=0.5)

Recommended Product List:
Product         | Rank Score
------------------------------
Phone A         | 0.5000
Phone B         | 0.5000
Phone C         | 0.5000
